Count gates in transpiled MCT circuit
===============================

In [1]:
import pytest
from pytest import raises
from circuits import SteaneCodeLogicalQubit
#from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit import execute, Aer
from qiskit.compiler import transpile
import numpy as np

from helper_functions import (
    flip_code_words,
    string_reverse,
    strings_AND_bitwise,
    string_ancilla_mask,
    correct_qubit,
    flip_code_words,
    count_valid_output_strings,
    mean_of_list,
    calculate_standard_error, 
    correct_qubit,
    print_time,
    get_parity_check_matrix,
    get_codewords
    )

SINGLE_GATE_SET = ['id', 'ry', 'rx']
TWO_GATE_SET = ['rxx']
BASIS_GATE_SET = SINGLE_GATE_SET + TWO_GATE_SET
TEST_X_QUBIT = 4
SHOTS = 10      #Number of shots to run    
SIMULATOR = Aer.get_backend('qasm_simulator')

parity_check_matrix = get_parity_check_matrix()
codewords = get_codewords()

SPACE = ' '
DATA_QUBITS = 7
NOISE = False

In [2]:
for MCT in {True, False}:    
    print_time()
    if MCT:
        qubit = SteaneCodeLogicalQubit(1, parity_check_matrix, codewords, extend_ancilla = False)
        print("Processing with MCT")
    else:
        qubit = SteaneCodeLogicalQubit(1, parity_check_matrix, codewords, extend_ancilla = True)
        print("Processing without MCT")
    qubit.set_up_logical_zero()
    qubit.barrier()
    index = 0
    qubit.force_X_error(index)   #force X error for testing
    #qubit.force_Z_error(1)   #force Z error for testing
    qubit.barrier()
    qubit.set_up_ancilla()
    qubit.barrier()
    if MCT:     #Multi Control Toffoli (MCT) gates
        qubit.correct_errors(mct = True)
    else:
        qubit.correct_errors()
    qubit.barrier()

    if NOISE:
        noise_model = get_noise(MEASURE_NOISE, SINGLE_GATE_DEPOLARISING, TWO_GATE_DEPOLARISING,
                           SINGLE_GATE_SET, TWO_GATE_SET)
    qubit.logical_measure_data()
    qubit.logical_measure_ancilla()
    qubit.barrier()
    qt = transpile(qubit, basis_gates = BASIS_GATE_SET)
    print('Transpiled gate statistics', qt.count_ops())
    print('Gate statistics before transpilation', qubit.count_ops())
    if NOISE:
        result = execute(qt, SIMULATOR, noise_model = noise_model, shots = SHOTS).result()
    else:
        result = execute(qt, SIMULATOR, shots = SHOTS).result()
    counts = result.get_counts(qt)
    print_time()
    print(f'Results for error introduced on qubit index {index}')
    print ('The simulated result after error correction are', counts)
    if MCT:
        count_valid, count_invalid, _ = count_valid_output_strings(counts, codewords, 2)
    else:
        count_valid, count_invalid, _ = count_valid_output_strings(counts, codewords, 3)
    error_rate = count_invalid / SHOTS
    print(f'The error rate is {error_rate:.4f}')
    print_time
    print()

Current Time = 09:42:19
Processing without MCT
Transpiled gate statistics OrderedDict([('rx', 387), ('ry', 183), ('rxx', 142), ('measure', 17), ('barrier', 12), ('reset', 6)])
Gate statistics before transpilation OrderedDict([('cx', 39), ('cz', 31), ('measure', 17), ('h', 15), ('reset', 13), ('barrier', 12), ('ccx', 12), ('x', 1)])
Current Time = 09:42:21
Results for error introduced on qubit index 0
The simulated result after error correction are {'0000 100 000 0000000': 2, '0000 100 000 0011110': 2, '0000 100 000 0101101': 1, '0000 100 000 0110011': 2, '0000 100 000 1001011': 1, '0000 100 000 1010101': 2}
The error rate is 0.0000

Current Time = 09:42:21
Processing with MCT
Transpiled gate statistics OrderedDict([('rx', 952), ('ry', 433), ('rxx', 312), ('measure', 13), ('barrier', 12), ('reset', 6)])
Gate statistics before transpilation OrderedDict([('x', 37), ('h', 29), ('cx', 20), ('mcx_gray', 14), ('reset', 13), ('measure', 13), ('barrier', 12), ('cz', 12)])
Current Time = 09:42:2